# NOTEBOOK FOURRE-TOUT pour scrips PlaMADE

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import os, re
import pandas as pd
import geopandas as gp
import zipfile
import Connexion_Transfert as ct
import Outils as O

In [ ]:
dossierSrc=r'C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees'
#dézipper tous les dossier de Gérard
for root,dirs, files in os.walk(dossierSrc) : 
    for f in files : 
        if f.endswith('.zip') : 
            cheminFichier=os.path.join(root, f)
            print(cheminFichier[:-4])
            try:
                with zipfile.ZipFile(cheminFichier) as z:
                    z.extractall(cheminFichier[:-4])
                    print(f"fichier extrait {cheminFichier[:-4]}")
            except:
                print(f"pb extraction sur dossier {cheminFichier[:-4]}")

## creer une bdd des fichiers geostandardises

In [ ]:
#pour traiter tous les fichiers d'un dossier général.suppose que la structure des tables a déjà été crée (par exemple avec l'import des des fichiers pouis truncate)
dossierSrc=r'C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees'
coupleFichierTable=(('N_ROUTIER_ALLURE','allure_national'),('N_ROUTIER_REVETEMENT','rvt_national'),('N_ROUTIER_ROUTE','route_national'),
                    ('N_ROUTIER_TRAFIC','trafic_national'),('N_ROUTIER_VITESSE','vts_national'))
#dossierSrc=r'C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees\Auvergne-Rhone-Alpes\Donnees_geostandardisees\Route01_v2_dec2020'
listEreur=[]
with ct.ConnexionBdd('local_PlaMADE', 'maison') as c :
    for root,dirs, files in os.walk(dossierSrc) : 
        for f in files :
            if f.endswith(('.shp', '.dbf')) : 
                print(os.path.join(root,f))
                if f.endswith('.shp') and 'N_ROUTIER_TRONCON' in f :
                    try :
                        ct.ogr2ogr_shp2pg(c.connstringOgr,os.path.join(root,f),
                                              schema='geostandardise_src', table='troncon_national',
                                              SRID=None,geotype='MULTILINESTRING', dims=3, creationMode='-append -update',encodageClient='UTF-8', version_simple=True)
                    except Exception as e: 
                        listEreur.append({'fichier': f, erreur : e})
                else : 
                    for fich,t in coupleFichierTable :
                        try : 
                            if fich in f.upper() and f.endswith('.dbf') : 
                                ct.ogr2ogr_shp2pg(c.connstringOgr,os.path.join(root,f),
                                                  schema='geostandardise_src', table=t, SRID=None,geotype=None, dims=None, creationMode='-append -update',encodageClient='UTF-8', requeteSql='', version_simple=True)
                            elif fich in f.upper() and f.endswith('.csv') :
                                df = pd.read_csv(os.path.join(root,f), 
                                                 keep_default_na=False)
                                df.columns=[c.lower() for c in df.columns]
                                df.drop(colonnesEnTrop,axis=1).to_sql(t,c.sqlAlchemyConn,'geostandardise_src', if_exists='append', index=False )
                        except Exception as e: 
                            listEreur.append({'fichier': f, 'erreur' : e})

In [ ]:
#si les donnees trafic, vts, allure , rvt sont en csv, le plus simple c'est pandas : 
dossierSrc=r'C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees\Bretagne'
coupleFichierTable=(('N_ROUTIER_ALLURE','allure_national'),('N_ROUTIER_REVETEMENT','rvt_national'),('N_ROUTIER_ROUTE','route_national'),
                    ('N_ROUTIER_TRAFIC','trafic_national'),('N_ROUTIER_VITESSE','vts_national'))
with ct.ConnexionBdd('local_PlaMADE', 'maison') as c :
    for root,dirs, files in os.walk(dossierSrc) : 
            for f in files :
                if f.endswith('.csv') :
                    for fich,t in coupleFichierTable :
                        if fich in f.upper() : 
                            try : 
                                print(os.path.join(root,f))
                                df = pd.read_csv(os.path.join(root,f), 
                                                keep_default_na=False)
                                df.columns=[c.lower() for c in df.columns]
                                dfref=pd.read_sql(f'select * from geostandardise_src.{t} limit 1',c.sqlAlchemyConn)
                                colonnesEnTrop=[c.lower() for c in df.columns if c not in dfref.columns]
                                df.drop(colonnesEnTrop,axis=1).to_sql(t,c.sqlAlchemyConn,'geostandardise_src', if_exists='append', index=False )
                            except Exception as e :
                                print(f'Erreur sur : {os.path.join(root,f)} : {e} ')

In [ ]:
#pour test sur un dept
with ct.ConnexionBdd('local_PlaMADE', 'boulot') as c :
    ct.ogr2ogr_shp2pg(c.connstringOgr,r'C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees\Bretagne\35\N_ROUTIER_TRONCON_L_035.shp',
                                              schema='geostandardise_src', table='troncon_national',
                                              SRID='2154',geotype='MULTILINESTRING', dims=3, creationMode='-append -update',encodageClient='UTF-8')

In [46]:
#sur un fichier
with ct.ConnexionBdd('local_PlaMADE', 'maison') as c :
    ct.ogr2ogr_shp2pg(c.connstringOgr,r'C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees\Auvergne_Rhone_Alpes\Donnees_geostandardisees\Route01_v2_dec2020\N_ROUTIER_ALLURE_001.dbf',
                                                  schema='geostandardise_src', table='allure_national', SRID=None,geotype=None, dims=None, creationMode='-append -update',encodageClient='UTF-8', requeteSql='', version_simple=True)


debut import fichier C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees\Auvergne_Rhone_Alpes\Donnees_geostandardisees\Route01_v2_dec2020\N_ROUTIER_ALLURE_001.dbf avec shape2pg à 17:05:54 
 avec commande ogr2ogr -append -update -f "postgreSQL" -lco "SCHEMA=geostandardise_src" -lco GEOMETRY_NAME=geom PG:"host=localhost dbname=plamade user=postgres password=postgres port=5432" C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees\Auvergne_Rhone_Alpes\Donnees_geostandardisees\Route01_v2_dec2020\N_ROUTIER_ALLURE_001.dbf -nln geostandardise_src.allure_national
Fait


In [ ]:
#transfert des deparyemenst
with ct.ConnexionBdd('local_PlaMADE', 'boulot') as c :
    for d in range(1,95) : 
        if d==20 : 
            continue
        dept=f'0{d}' if d<10 else str(d)
        cheminDept = fr'Y:\REF_GEO\BD_Topo\D{dept}\ED19\SHP\1_DONNEES_LIVRAISON\ADMINISTRATIF\N_DEPARTEMENT_TOPO_0{dept}.shp'
        if d==1 : 
            ct.ogr2ogr_shp2pg(c.connstringOgr,cheminDept,
                                                  schema='geostandardise_src', table='dept', SRID='2154',geotype='MULTIPOLYGON', dims=2, creationMode='',encodageClient='UTF-8', requeteSql='')
        else :
            ct.ogr2ogr_shp2pg(c.connstringOgr,cheminDept,
                                                  schema='geostandardise_src', table='dept', SRID='2154',geotype='MULTIPOLYGON', dims=2, creationMode='-append -update',encodageClient='UTF-8', requeteSql='')